# ICEfinder parser

## Preparation

In [1]:
import pandas as pd
from pathlib import Path
import csv
import gffutils

In [59]:
project_path=Path().resolve().parent
#input_path=project_path / "results" / "intermediate" / "benchmarking" / phaster_raw"
result_path=project_path / "results" / "intermediate" / "benchmarking" / "icefinder_parsed"
gff_path=project_path / "data" / "legen_v4_dereplicated_gffs"

In [3]:
genome="GCA_902163985.1"

## Generate ICEfinder output file

In [40]:
icefinder=pd.DataFrame(columns=['REGION', 'REGION_LENGTH', 'NAME', 'START', 'END', 'CONTIG'])

In [41]:
icefinder

,REGION,REGION_LENGTH,NAME,START,END,CONTIG


In [43]:
icefinder=icefinder.append({'REGION':1, 'REGION_LENGTH':77921, 'NAME':"Putative IME", 'START':42387, 'END':120307, 'CONTIG':"CABHBK010000001.1"}, ignore_index=True)
#icefinder=icefinder.append({'REGION':1, 'REGION_LENGTH':77921, 'NAME':"Putative IME", 'START':42387, 'END':120307}, ignore_index=True)




In [44]:
icefinder

,REGION,REGION_LENGTH,NAME,START,END,CONTIG
0,1,77921,Putative IME,42387,120307,CABHBK010000001.1


## Location to gene nr

In [45]:
def get_genes_multi_contig(df, start, stop, contig):
    return df[(df.start>=start) & (df.end<=stop) & (df.contig==contig)]

In [9]:
def get_genes(df, start, stop):
    return df[(df.start>=start) & (df.end<=stop)]

In [50]:
def add_start_end_gene(genome, icefinder):
    gff= str(str(gff_path) + "/" + genome + ".gff")
    # generate a database
    gffutils.create_db(gff, str(genome + "_db"))
    db = gffutils.FeatureDB(dbfn=str(genome + "_db"))
    # generate a dataframe
    df=pd.DataFrame(columns = ['contig', 'ID', 'start', 'end', 'strand'])
    # fill up the dataframe, using the database
    query = db.execute("select seqid,start,end,strand,attributes from features where featuretype = 'CDS'")
    result = query.fetchall()

    for each in result:
        df=df.append({'ID':json.loads(each['attributes'])['ID'][0], 'contig':each['seqid'], 'end':each['end'], 'start':each['start'], 'strand':each['strand']}, ignore_index=True)
    
    # add start genes
    start_genes=[]
    for index, row in icefinder.iterrows():
        genes=get_genes(df, int(row.START), int(row.END))
        start_genes.append(get_genes_multi_contig(df, int(row.START), int(row.END), row.CONTIG).index[0])
    icefinder['START_GENE']=start_genes
    
    # add end genes
    end_genes=[]
    for index, row in icefinder.iterrows():
        end_genes.append(get_genes_multi_contig(df, int(row.START), int(row.END), row.CONTIG).index[-1])
    icefinder['END_GENE']=end_genes
    db=str(genome + "_db")
    !rm $db
    return icefinder
    
    

In [51]:
add_start_end_gene(genome, icefinder)

,REGION,REGION_LENGTH,NAME,START,END,CONTIG,START_GENE,END_GENE
0,1,77921,Putative IME,42387,120307,CABHBK010000001.1,39,113


In [12]:
db=str(genome + "_db")
!rm $db

In [63]:
! mkdir $result_path/$genome